Note: Multiplier has been taken as it is. For averages and ratios it doesn't affect athe analysis. To use absolute population values, use multiplier/100 instead

# Importing libraries

In [1]:
import pandas as pd
import numpy as np
import pathlib
import pickle
from functools import reduce

In [2]:
pd.set_option('display.max_columns', None)

# Importing datasets

In [ ]:
# Checki|ng paths
pathlib.Path("../../HCES 2022-23/Python implementation/Codes/Population based MPCE/MPCE_1Jan24.pkl").resolve().exists()

True

In [60]:
# Reading HH-wise MPCE
path_mpce = "../../HCES 2022-23/Python implementation/Codes/Population based MPCE/MPCE_1Jan24.pkl"
df_mpce = pd.read_pickle(path_mpce)
df_mpce = df_mpce[['hh_size', 'sector', 'mpce', 'decile']]
display(df_mpce.head())

## Level 01 file
path_lvl01 = "../../HCES 2022-23/Dta raw files/level_01.dta"
df_lvl01 = pd.read_stata(path_lvl01)
df_lvl01 = df_lvl01.set_index('hhid')
df_lvl01 = df_lvl01[['state']]
display(df_lvl01.head())

## Level 03 file
path_lvl03 = "../../HCES 2022-23/Dta raw files/level_03.dta"
df_lvl03 = pd.read_stata(path_lvl03)
df_lvl03 = df_lvl03.set_index('hhid')
df_lvl03 = df_lvl03[['source_cooking', 'source_lighting','multiplier']]
display(df_lvl03.head())

## Level 07 file
path_lvl07 = "../../HCES 2022-23/Dta raw files/level_07.dta"
df_lvl07 = pd.read_stata(path_lvl07)
df_lvl07 = df_lvl07.set_index('hhid')
df_lvl07 = df_lvl07[['receieved_subsidy_lpg', 'num_subsidized_lpg']]
display(df_lvl07.head())

## Level 08 file
path_lvl08 = "../../HCES 2022-23/Dta raw files/level_08.dta"
df_lvl08 = pd.read_stata(path_lvl08)
df_lvl08 = df_lvl08.loc[df_lvl08['item_code'].isin([331,338])]

,hh_size,sector,mpce,decile
hhid,,,,
31000301,1.0,2,19675.64,10
31000302,5.0,2,8059.52,8
31000303,5.0,2,7628.02,8
31000304,2.0,2,6522.62,7
31000305,1.0,2,6765.77,7


,state
hhid,
65556201,1
65556301,1
65556302,1
65556303,1
65556304,1


,source_cooking,source_lighting,multiplier
hhid,,,
31000301,2,1,96498
31000302,2,1,96498
31000303,2,1,96498
31000304,2,1,96498
31000305,2,1,96498


,receieved_subsidy_lpg,num_subsidized_lpg
hhid,,
31000301,2,NaN
31000302,1,1.0
31000303,1,1.0
31000304,1,1.0
31000305,2,NaN


# Data wrangling

In [61]:
# Reshaping level 08 file for firewood and LPG

## Pivoting
df_lvl08 = df_lvl08.pivot_table(index = 'hhid', columns = 'item_code', values = ['cons_total_qty','cons_total_value', 'cons_home_qty', 'cons_home_value' ])

## Converting columns to single level index
df_lvl08.columns = ["_".join(map(str, x)) for x in df_lvl08.columns]

## Changing column names
df_lvl08 = df_lvl08.rename(columns = {'cons_total_qty_331': 'firewood_qty', 'cons_total_qty_338': 'lpg_qty',
                               'cons_total_value_331': 'firewood_value', 'cons_total_value_338': 'lpg_value',
                               'cons_home_qty_331': 'firewood_home_qty', 'cons_home_value_331': 'firewood_home_value',
                               'cons_home_qty_338': 'lpg_home_qty', 'cons_home_value_338': 'lpg_home_value'})
df_lvl08.head()

# Adding market quantity and value
df_lvl08 = df_lvl08.fillna(0)
df_lvl08[['firewood_mkt_qty', 'firewood_mkt_value']] = df_lvl08[['firewood_qty', 'firewood_value' ]].values - df_lvl08[['firewood_home_qty', 'firewood_home_value' ]].values
df_lvl08[['lpg_mkt_qty', 'lpg_mkt_value']] = df_lvl08[['lpg_qty', 'lpg_value' ]].values - df_lvl08[['lpg_home_qty', 'lpg_home_value' ]].values

df_lvl08.head()

,firewood_home_qty,lpg_home_qty,firewood_home_value,lpg_home_value,firewood_qty,lpg_qty,firewood_value,lpg_value,firewood_mkt_qty,firewood_mkt_value,lpg_mkt_qty,lpg_mkt_value
hhid,,,,,,,,,,,,
31000301,0.0,0.0,0.0,0.0,0.0,2.0,0.0,240.0,0.0,0.0,2.0,240.0
31000302,0.0,0.0,0.0,0.0,0.0,8.0,0.0,640.0,0.0,0.0,8.0,640.0
31000303,0.0,0.0,0.0,0.0,0.0,9.0,0.0,720.0,0.0,0.0,9.0,720.0
31000304,0.0,0.0,0.0,0.0,0.0,5.0,0.0,400.0,0.0,0.0,5.0,400.0
31000305,0.0,0.0,0.0,0.0,0.0,4.0,0.0,320.0,0.0,0.0,4.0,320.0


In [62]:
# Merging all files

## Checking shapes
print(f"Shape of MPCE file: {df_mpce.shape}")
print(f"Shape of level 01 file: {df_lvl01.shape}")
print(f"Shape of level 03 file: {df_lvl03.shape}")
print(f"Shape of level 07 file: {df_lvl07.shape}")
print(f"Shape of level 08 file: {df_lvl08.shape}")

## Defining merge function
def merge_df (left_df, right_df):
    df = pd.merge(left_df, right_df, how = 'outer', left_index=True, right_index=True, indicator=True)
    print(f"\nSummary from merging dataframes")
    display(df['_merge'].value_counts())
    df = df.drop(labels = '_merge', axis = 1)
    return df.copy()

## Merging
df = reduce(merge_df, [df_mpce, df_lvl01, df_lvl03, df_lvl07, df_lvl08])

Shape of MPCE file: (261746, 4)
Shape of level 01 file: (261746, 1)
Shape of level 03 file: (261746, 3)
Shape of level 07 file: (261746, 2)
Shape of level 08 file: (252955, 12)

Summary from merging dataframes


_merge
both          261746
left_only          0
right_only         0
Name: count, dtype: int64


Summary from merging dataframes


_merge
both          261746
left_only          0
right_only         0
Name: count, dtype: int64


Summary from merging dataframes


_merge
both          261746
left_only          0
right_only         0
Name: count, dtype: int64


Summary from merging dataframes


_merge
both          252955
left_only       8791
right_only         0
Name: count, dtype: int64

In [63]:
# Renaming column values
df['sector'] = df['sector'].replace({1: 'rural', 2: 'urban'})
df['state'] = df['state'].replace({1: 'Jammu & Kashmir', 2: 'Himachal Pradesh', 3: 'Punjab', 4: 'Chandigarh(U.T.)', 5: 'Uttrakhand', 6: 'Haryana', 7: 'Delhi', 8: 'Rajasthan', 9: 'Uttar Pradesh', 10: 'Bihar', 11: 'Sikkim', 12: 'Arunachal Pradesh', 13: 'Nagaland', 14: 'Manipur', 15: 'Mizoram', 16: 'Tripura', 17: 'Meghalaya', 18: 'Assam', 19: 'West Bengal', 20: 'Jharkhand', 21: 'Odisha', 22: 'Chattisgarh', 23: 'Madhya Pradesh', 24: 'Gujarat', 25: 'Dadra & Nagar Haveli', 27: 'Maharashtra', 28: 'Andhra Pradesh', 29: 'Karnataka', 30: 'Goa', 31: 'Lakshadweep (U.T.)', 32: 'Kerala', 33: 'Tamilnadu', 34: 'Puducherry (U.T.)', 35: 'A and N Islands (U.T.)', 36: 'Telangana', 37: 'Ladakh (U.T.)'})
df['source_cooking'] = df['source_cooking'].replace({1: "firewood and chips", 2: "LPG", 3: "other natural gas", 4: "dung cake", 5: "kerosene", 6: "coke, coal", 7: "gobar gas", 8: "other biogas", 9: "others", 10: "charcoal", 11: "electricity", 12: "no cooking arrangement"})
df['receieved_subsidy_lpg'] = df['receieved_subsidy_lpg'].replace({1: 'yes', 2: 'no'})

# Analysis

## Primary cooking source

In [88]:
# Across India

## Calculating
trial = df.groupby(['source_cooking'], as_index=False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum()/100,
     'sample_size': table.shape[0]}
), include_groups = False)
display(trial)

,source_cooking,popn_size,sample_size
0,LPG,1.592795e+08,160329.0
1,charcoal,2.740225e+04,85.0
2,"coke, coal",1.018071e+06,1156.0
3,dung cake,6.953348e+06,5076.0
4,electricity,3.592216e+05,431.0
5,firewood and chips,1.064971e+08,86947.0
6,gobar gas,1.365552e+05,105.0
7,kerosene,1.955983e+05,206.0
8,no cooking arrangement,6.284528e+06,3801.0
9,other biogas,2.135539e+04,15.0


In [89]:
# Primary cooking source (Rural vs Urban)

## Calculating
trial = df.groupby(['sector','source_cooking']).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum()/100,
     'sample_size': table.shape[0]}
), include_groups = False)
trial = trial.reset_index()

## Pivoting
trial = trial.pivot_table(values = 'popn_size', index = ['sector'], columns = 'source_cooking')
display(trial)

source_cooking,LPG,charcoal,"coke, coal",dung cake,electricity,firewood and chips,gobar gas,kerosene,no cooking arrangement,other biogas,other natural gas,others
sector,,,,,,,,,,,,
rural,83665123.16,10601.30,550118.89,6734277.75,92907.19,1.006922e+08,134079.87,34084.25,2115302.73,16549.81,242157.03,607965.58
urban,75614399.13,16800.95,467952.01,219070.51,266314.38,5.804880e+06,2475.36,161514.07,4169225.59,4805.58,2433651.78,341921.24


In [90]:
# Primary cooking source (Rural vs Urban by decile)

## Calculating
trial = df.groupby(['sector','decile','source_cooking'], as_index=False, observed=False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum()/100,
     'sample_size': table.shape[0]}
), include_groups = False)


## Pivoting
trial = trial.pivot_table (values = 'popn_size', index = ['sector', 'source_cooking'], columns = 'decile', observed = False)
display(trial)


decile                                   1            2            3  \
sector source_cooking                                                  
rural  LPG                      1739901.72   3074245.05   4237089.22   
       charcoal                        NaN       267.17      1160.18   
       coke, coal                 49532.82     61497.18     48527.60   
       dung cake                1111775.50    865310.48    855947.48   
       electricity                 1238.45      4391.21      5553.82   
       firewood and chips      11543687.29  11740581.30  11459039.80   
       gobar gas                   1229.12      5650.61      3969.68   
       kerosene                    1023.76      2253.16      5854.03   
       no cooking arrangement    220294.78     67035.82     58540.34   
       other biogas                    NaN          NaN          NaN   
       other natural gas           1438.76      3800.08      2456.53   
       others                     32542.50     68938.97     57472.99   
urban  LPG                      3879098.59   5351826.77   6209002.69   
       charcoal                    6174.37       825.72       713.41   
       coke, coal                173665.06     96978.89     72482.69   
       dung cake                  84277.11     41442.91     37495.10   
       electricity                10393.72     12230.64      9829.89   
       firewood and chips       1748448.41   1149805.87    816618.30   
       gobar gas                   1243.28          NaN          NaN   
       kerosene                   26498.91     19968.46     15372.80   
       no cooking arrangement     49019.76    109820.97     96123.92   
       other biogas                    NaN          NaN      1002.72   
       other natural gas           5864.74     24091.57     66224.22   
       others                     19784.57     12672.52     18184.23   

decile                                   4            5            6  \
sector source_cooking                                                  
rural  LPG                      5167786.33   6582871.14   7927863.46   
       charcoal                        NaN       402.00       576.09   
       coke, coal                 58226.42     67155.82     79289.90   
       dung cake                 659592.54    657416.54    587036.34   
       electricity                 5490.57      8426.65      6634.99   
       firewood and chips      11468298.11  10673734.35  10482914.27   
       gobar gas                   6416.33      6364.06     12697.61   
       kerosene                    2632.24      2047.52      2806.35   
       no cooking arrangement     79744.38    116570.42     96128.68   
       other biogas                1461.83          NaN          NaN   
       other natural gas           3493.56       596.48      5885.92   
       others                     54941.38     85850.61     55629.90   
urban  LPG                      6775525.84   7339305.68   7974095.07   
       charcoal                    1833.76        49.82      1418.90   
       coke, coal                 45507.56     29241.38     22816.78   
       dung cake                  20829.57     12498.71      8898.19   
       electricity                 8169.01     10027.46      9110.02   
       firewood and chips        598667.70    456827.29    368287.34   
       gobar gas                       NaN          NaN          NaN   
       kerosene                   10200.16     21942.75     13764.01   
       no cooking arrangement     79822.44     87859.94     96836.30   
       other biogas                1119.49      2683.37          NaN   
       other natural gas          78290.40    123337.49    169708.16   
       others                     25645.39     12508.77     17242.40   

decile                                  7            8            9  \
sector source_cooking                                                 
rural  LPG                     9545674.10  11459344.44  14392834.98   
       charcoal                   2423.77

## LPG subsidy - Last three months

In [91]:
# Across India
trial = df.groupby(['receieved_subsidy_lpg'], as_index=False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum()/100,
     'sample_size': table.shape[0]}
), include_groups = False)
display(trial)

,receieved_subsidy_lpg,popn_size,sample_size
0,no,2.212992e+08,203277.0
1,yes,6.309924e+07,58469.0


In [92]:
# LPG subsidy (Rural vs Urban)
trial = df.groupby(['sector','receieved_subsidy_lpg'], as_index=False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum()/100,
     'sample_size': table.shape[0]}
), include_groups = False)

# Pivoting 
trial = trial.pivot_table(values = 'popn_size', index = 'receieved_subsidy_lpg', columns = 'sector')
display(trial)

sector,rural,urban
receieved_subsidy_lpg,,
no,1.533033e+08,67995834.14
yes,4.159206e+07,21507176.24


In [93]:
# LPG subsidy (Rural vs Urban by decile)
trial = df.groupby(['sector','decile','receieved_subsidy_lpg'], as_index=False, observed=False).apply(lambda table: pd.Series(
    {'wt_sum': table['multiplier'].sum()/100,
     'sample_size': table.shape[0]}
), include_groups = False)

# Pivoting
trial = trial.pivot_table(values = 'wt_sum', index = ['sector','receieved_subsidy_lpg'], columns = 'decile', observed = False)
display(trial)

decile                                  1            2            3  \
sector receieved_subsidy_lpg                                          
rural  no                     13453380.33  13515547.37  13752204.28   
       yes                     1249284.37   2378423.66   2983407.39   
urban  no                      4426188.52   4898582.50   5283759.91   
       yes                     1578280.00   1921081.82   2059290.06   

decile                                  4            5            6  \
sector receieved_subsidy_lpg                                          
rural  no                     14294632.13  14475754.83  14981770.73   
       yes                     3213451.56   3725680.76   4275692.78   
urban  no                      5578999.39   5975515.33   6431200.56   
       yes                     2066611.93   2120767.33   2250976.61   

decile                                  7            8            9  \
sector receieved_subsidy_lpg                                          
rural  no                     15337654.61  16234911.21  16798533.56   
       yes                     4804034.73   5311479.80   6326242.24   
urban  no                      6942476.83   7553499.30   8458141.23   
       yes                     2251613.57   2399773.97   2613066.72   

decile                                 10  
sector receieved_subsidy_lpg               
rural  no                     20458956.79  
       yes                     7324365.72  
urban  no                     12447470.57  
       yes                     2245714.23

## LPG susbsidised cylinder - Last 3 months

In [94]:
# Across India

## Filtering for people who recieved subsidy
trial = df.copy()
trial = trial.loc[trial['receieved_subsidy_lpg'] == 'yes']

# Calculation
trial = trial.groupby('num_subsidized_lpg', as_index = False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum()/100,
     'sample_size': table.shape[0]}
), include_groups = False)
display(trial)

,num_subsidized_lpg,popn_size,sample_size
0,1.0,39005158.61,34416.0
1,2.0,17504663.36,16881.0
2,3.0,6589417.28,7172.0


In [71]:
# Rural vs Urban

## Filtering for people who recieved subsidy
trial = df.copy()
trial = trial.loc[trial['receieved_subsidy_lpg'] == 'yes']

# Calculation
trial = trial.groupby(['sector','num_subsidized_lpg']).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum(),
     'sample_size': table.shape[0]}
), include_groups = False)
display(trial)

popn_size  sample_size
sector num_subsidized_lpg                         
rural  1.0                 2831518859        21606
       2.0                 1003612412         7831
       3.0                  324075030         2648
urban  1.0                 1068997002        12810
       2.0                  746853924         9050
       3.0                  334866698         4524

In [95]:
# Rural vs Urban across deciles

## Filtering for people who recieved subsidy
trial = df.copy()
trial = trial.loc[trial['receieved_subsidy_lpg'] == 'yes']

## Calculation
trial = trial.groupby(['sector', 'decile', 'num_subsidized_lpg'], observed = False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum()/100,
     'sample_size': table.shape[0]}
), include_groups = False)
trial = trial.reset_index()

# Pivotting
trial = trial.pivot_table(values = 'popn_size', index = ['sector', 'num_subsidized_lpg'], columns = 'decile',observed = False)
display(trial)

decile                             1           2           3           4  \
sector num_subsidized_lpg                                                  
rural  1.0                 973055.00  1785819.65  2192972.80  2326765.10   
       2.0                 212154.79   450258.01   599893.60   646059.08   
       3.0                  64074.58   142346.00   190540.99   240627.38   
urban  1.0                 777079.59   911106.93  1040670.25  1032303.37   
       2.0                 526772.97   663642.20   670856.49   700942.92   
       3.0                 274427.44   346332.69   347763.32   333365.64   

decile                              5           6           7           8  \
sector num_subsidized_lpg                                                   
rural  1.0                 2610463.05  2995328.10  3154317.68  3542789.78   
       2.0                  845269.79   956959.82  1206382.23  1383012.53   
       3.0                  269947.92   323404.86   443334.82   385677.49   
urban  1.0                 1042736.86  1093006.37  1091543.27  1191317.65   
       2.0                  758584.31   806912.39   801060.84   835256.53   
       3.0                  319446.16   351057.85   359009.46   373199.79   

decile                              9          10  
sector num_subsidized_lpg                          
rural  1.0                 4092456.67  4641220.76  
       2.0                 1709296.53  2026837.74  
       3.0                  524489.04   656307.22  
urban  1.0                 1350158.28  1160047.45  
       2.0                  916187.12   788323.47  
       3.0                  346721.32   297343.31

## Average quantity and value of TOTAL LPG and firewood consumed per capita

* Values are reported for past 30 days

In [73]:
# Across India

## Duplicate df
trial = df.copy()

## Sumproduct of consumption
value_mult = pd.Series({
    'lpg_qty': (trial['lpg_qty'] * trial['multiplier']).sum(),
    'lpg_value':(trial['lpg_value'] * trial['multiplier']).sum(),
    'firewood_qty': (trial['firewood_qty'] * trial['multiplier']).sum(),
    'firewood_value':(trial['firewood_value'] * trial['multiplier']).sum()
}).round(2)

## Sumproduct of hh size
hh_size_mult = (trial['hh_size']*trial['multiplier']).sum()

# Weighted averages
trial = value_mult/hh_size_mult
display(trial)

lpg_qty             1.473315
lpg_value         117.535183
firewood_qty       11.037265
firewood_value     51.857151
dtype: float64

In [74]:
# Rural vs Urban

# Duplicate df and replace Nan values
trial = df.copy()
trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']] = trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']].fillna(0)


# Sumproduct of consumption
value_mult = trial.groupby('sector').apply(lambda table: pd.Series({
    'wt_lpg_qty': (table['lpg_qty']*table['multiplier']).sum(),
    'wt_lpg_value': (table['lpg_value']*table['multiplier']).sum(),
    'wt_firewod_qty': (table['firewood_qty']*table['multiplier']).sum(),
    'wt_firewood_value': (table['firewood_value']*table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby('sector').apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0)
display(trial)

,wt_lpg_qty,wt_lpg_value,wt_firewod_qty,wt_firewood_value
sector,,,,
rural,1.127155,90.514766,14.474689,67.810782
urban,2.348439,185.845210,2.347152,11.524946


In [247]:
# Rural vs Urban across deciles

# Duplicate df and replace Nan values
trial = df.copy()
trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']] = trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']].fillna(0)


# Sumproduct of consumption
value_mult = trial.groupby(['sector', 'decile'], observed=False).apply(lambda table: pd.Series({
    'wt_lpg_qty': (table['lpg_qty']*table['multiplier']).sum(),
    'wt_lpg_value': (table['lpg_value']*table['multiplier']).sum(),
    'wt_firewod_qty': (table['firewood_qty']*table['multiplier']).sum(),
    'wt_firewood_value': (table['firewood_value']*table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby(['sector', 'decile'], observed=False).apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0).round(2)
display(trial)

wt_lpg_qty  wt_lpg_value  wt_firewod_qty  wt_firewood_value
sector decile                                                             
rural  1             0.29         23.72           15.31              67.78
       2             0.52         41.90           16.06              72.38
       3             0.68         54.98           15.78              72.80
       4             0.80         64.15           15.65              73.38
       5             0.97         77.81           15.17              71.79
       6             1.13         90.41           14.99              71.43
       7             1.30        104.57           14.19              67.55
       8             1.51        120.98           13.87              67.05
       9             1.78        142.92           12.60              60.58
       10            2.29        183.72           11.13              53.38
urban  1             1.38        109.59            6.13              29.79
       2             1.84        145.65            4.30              20.93
       3             2.08        164.73            3.16              15.70
       4             2.23        176.09            2.56              12.93
       5             2.35        185.86            2.06               9.91
       6             2.51        198.28            1.65               8.40
       7             2.60        205.14            1.35               6.83
       8             2.72        215.05            0.92               4.39
       9             2.84        225.40            0.76               3.83
       10            2.93        232.67            0.58               2.54

In [11]:
df.head()

,hh_size,sector,mpce,decile,state,source_cooking,multiplier,receieved_subsidy_lpg,num_subsidized_lpg,firewood_home_qty,lpg_home_qty,firewood_home_value,lpg_home_value,firewood_qty,lpg_qty,firewood_value,lpg_value,firewood_mkt_qty,firewood_mkt_value,lpg_mkt_qty,lpg_mkt_value
hhid,,,,,,,,,,,,,,,,,,,,,
31000301,1.0,urban,19675.64,10,Andhra Pradesh,LPG,96498,no,NaN,0.0,0.0,0.0,0.0,0.0,2.0,0.0,240.0,0.0,0.0,2.0,240.0
31000302,5.0,urban,8059.52,8,Andhra Pradesh,LPG,96498,yes,1.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,640.0,0.0,0.0,8.0,640.0
31000303,5.0,urban,7628.02,8,Andhra Pradesh,LPG,96498,yes,1.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,720.0,0.0,0.0,9.0,720.0
31000304,2.0,urban,6522.62,7,Andhra Pradesh,LPG,96498,yes,1.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,400.0,0.0,0.0,5.0,400.0
31000305,1.0,urban,6765.77,7,Andhra Pradesh,LPG,96498,no,NaN,0.0,0.0,0.0,0.0,0.0,4.0,0.0,320.0,0.0,0.0,4.0,320.0


In [16]:
# Rural vs Urban across states

# Duplicate df and replace Nan values
trial = df.copy()


# Sumproduct of consumption
value_mult = trial.groupby(['sector', 'state'], observed = False).apply(lambda table: pd.Series({
    'wt_lpg_qty': (table['lpg_qty'] * table['multiplier']).sum(),
    'wt_lpg_value':(table['lpg_value'] * table['multiplier']).sum(),
    'wt_firewood_qty': (table['firewood_qty'] * table['multiplier']).sum(),
    'wt_firewood_value': (table['firewood_value'] * table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby(['sector', 'state'], observed = False).apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0)
display(trial)

wt_lpg_qty  wt_lpg_value  wt_firewood_qty  \
sector state                                                               
rural  A and N Islands (U.T.)    2.655244    217.624077         2.361476   
       Andhra Pradesh            1.895074    152.515515         6.706925   
       Arunachal Pradesh         0.968536     79.500800        25.564439   
       Assam                     0.967616     78.142976        16.955429   
       Bihar                     0.854335     72.119167        10.485264   
...                                   ...           ...              ...   
urban  Telangana                 2.014468    168.378900         0.260950   
       Tripura                   2.711926    239.896157         7.097235   
       Uttar Pradesh             2.300679    181.358964         1.678701   
       Uttrakhand                2.824488    221.467120         1.627196   
       West Bengal               2.640153    208.327769         4.619056   

                               wt_firewood_value  
sector state                                      
rural  A and N Islands (U.T.)          14.484645  
       Andhra Pradesh                  27.148969  
       Arunachal Pradesh              140.625085  
       Assam                           84.168833  
       Bihar                           69.381512  
...                                          ...  
urban  Telangana                        1.722246  
       Tripura                         45.793740  
       Uttar Pradesh                    8.900367  
       Uttrakhand                       6.784211  
       West Bengal                     23.653839  

[72 rows x 4 columns]

In [18]:
trial[trial.index.get_level_values(0) == 'urban']

wt_lpg_qty  wt_lpg_value  wt_firewood_qty  \
sector state                                                               
urban  A and N Islands (U.T.)    3.454023    279.267002         0.017689   
       Andhra Pradesh            2.204471    177.001798         0.708256   
       Arunachal Pradesh         2.339209    189.140116         6.947354   
       Assam                     2.635281    212.195192         2.027767   
       Bihar                     2.155083    182.423019         2.832444   
       Chandigarh(U.T.)          2.251030    176.912552         0.000000   
       Chattisgarh               2.242680    187.274975         4.455340   
       Dadra & Nagar Haveli      2.798040    226.534882         1.655415   
       Delhi                     2.534172    193.861395         0.009241   
       Goa                       2.835417    219.794394         2.317326   
       Gujarat                   1.661163    127.528652         1.130059   
       Haryana                   2.719349    211.462781         0.849601   
       Himachal Pradesh          2.950742    241.693747         3.921540   
       Jammu & Kashmir           2.676464    218.977621         1.514679   
       Jharkhand                 1.868535    154.081541         3.148507   
       Karnataka                 2.285396    180.555801         1.990440   
       Kerala                    2.381659    186.121426        11.439337   
       Ladakh (U.T.)             1.918624    173.322380         0.428724   
       Lakshadweep (U.T.)        1.647684    183.010257         8.075612   
       Madhya Pradesh            1.979181    156.661691         2.185678   
       Maharashtra               2.496376    193.512111         0.886770   
       Manipur                   2.004262    172.411538         0.659906   
       Meghalaya                 2.139876    177.713494         4.820533   
       Mizoram                   3.871720    274.602860         0.285066   
       Nagaland                  2.155546    182.938703        22.278374   
       Odisha                    2.104534    168.754428        12.153099   
       Puducherry (U.T.)         2.797451    222.453871         1.598193   
       Punjab                    2.892980    230.333942         1.153475   
       Rajasthan                 2.172480    167.857704         3.408799   
       Sikkim                    2.827130    262.780401         0.316249   
       Tamilnadu                 2.736947    219.106896         0.902824   
       Telangana                 2.014468    168.378900         0.260950   
       Tripura                   2.711926    239.896157         7.097235   
       Uttar Pradesh             2.300679    181.358964         1.678701   
       Uttrakhand                2.824488    221.467120         1.627196   
       West Bengal               2.640153    208.327769         4.619056   

                               wt_firewood_value  
sector state                                      
urban  A and N Islands (U.T.)           0.106132  
       Andhra Pradesh                   4.058877  
       Arunachal Pradesh               41.052123  
       Assam                           12.509006  
       Bihar                           19.400803  
       Chandigarh(U.T.)                 0.000000  
       Chattisgarh                     23.096648  
       Dadra & Nagar Haveli             8.262545  
       Delhi                            0.146277  
       Goa                             10.777899  
       Gujarat                          6.811307  
       Haryana                          5.052014  
       Himachal Pradesh                16.157218  
       Jammu & Kashmir                  6.719560  
       Jharkhand                       14.312353  
       Karnataka                       10.012385  
       Kerala                          43.573286  
       Ladakh (U.T.)                    3.372922  
       Lakshadweep (U.T.)              26.606090  
       Madhya Pradesh                  10.029168  
       Maharashtra  

## Average quantity and value of MARKET firewood consumed per capita

In [255]:
# Across India

## Duplicate df
trial = df.copy()

## Sumproduct of consumption
value_mult = pd.Series({
    'firewood_mkt_qty': (trial['firewood_mkt_qty'] * trial['multiplier']).sum(),
    'firewood_mkt_value':(trial['firewood_mkt_value'] * trial['multiplier']).sum()
}).round(2)

## Sumproduct of hh size
hh_size_mult = (trial['hh_size']*trial['multiplier']).sum()

# Weighted averages
trial = value_mult/hh_size_mult
display(trial)

firewood_mkt_qty       7.677946
firewood_mkt_value    36.449999
dtype: float64

In [256]:
# Rural vs Urban

# Duplicate df and replace Nan values
trial = df.copy()


# Sumproduct of consumption
value_mult = trial.groupby('sector').apply(lambda table: pd.Series({
    'wt_firewood_mkt_qty': (table['firewood_mkt_qty'] * table['multiplier']).sum(),
    'wt_firewood_mkt_value':(table['firewood_mkt_value'] * table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby('sector').apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0)
display(trial)

,wt_firewood_mkt_qty,wt_firewood_mkt_value
sector,,
rural,9.986364,47.110268
urban,1.842060,9.499887


In [259]:
# Rural vs Urban across deciles

# Duplicate df and replace Nan values
trial = df.copy()


# Sumproduct of consumption
value_mult = trial.groupby(['sector', 'decile'], observed = False).apply(lambda table: pd.Series({
    'wt_firewood_mkt_qty': (table['firewood_mkt_qty'] * table['multiplier']).sum(),
    'wt_firewood_mkt_value':(table['firewood_mkt_value'] * table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby(['sector', 'decile'], observed = False).apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0)
display(trial)

wt_firewood_mkt_qty  wt_firewood_mkt_value
sector decile                                            
rural  1                 12.019989              53.076220
       2                 11.860949              53.813415
       3                 11.320030              52.788773
       4                 11.180783              52.685019
       5                 10.659816              50.925218
       6                 10.225955              48.933431
       7                  9.329608              44.787533
       8                  9.058715              44.088838
       9                  7.701046              37.614000
       10                 6.506182              32.387946
urban  1                  5.301535              26.299058
       2                  3.505134              17.804532
       3                  2.401369              12.651682
       4                  1.930987              10.328211
       5                  1.585764               7.959233
       6                  1.203891               6.656743
       7                  0.990697               5.384606
       8                  0.592732               3.191518
       9                  0.532468               2.946313
       10                 0.375457               1.774245

In [40]:
# Rural vs Urban across states

# Duplicate df and replace Nan values
trial = df.copy()


# Sumproduct of consumption
value_mult = trial.groupby(['sector', 'state'], observed = False).apply(lambda table: pd.Series({
    'wt_firewood_mkt_qty': (table['firewood_mkt_qty'] * table['multiplier']).sum(),
    'wt_firewood_mkt_value':(table['firewood_mkt_value'] * table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby(['sector', 'state'], observed = False).apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0)
display(trial)

wt_firewood_mkt_qty  wt_firewood_mkt_value
sector state                                                             
rural  A and N Islands (U.T.)             2.286046              13.300046
       Andhra Pradesh                     6.404175              25.909515
       Arunachal Pradesh                 19.685340             112.403346
       Assam                              8.646849              46.328774
       Bihar                              7.647358              49.365340
...                                            ...                    ...
urban  Telangana                          0.247391               1.633835
       Tripura                            5.669061              37.511397
       Uttar Prdesh                       1.403836               7.702412
       Uttrakhand                         1.446670               6.368216
       West Bengal                        4.026102              20.790609

[72 rows x 2 columns]

In [41]:
trial[trial.index.get_level_values(0) == 'urban']

wt_firewood_mkt_qty  wt_firewood_mkt_value
sector state                                                             
urban  A and N Islands (U.T.)             0.017689               0.106132
       Andhra Pradesh                     0.691379               3.956743
       Arunachal Pradesh                  5.820248              36.357900
       Assam                              1.559906               9.377244
       Bihar                              2.334052              15.699532
       Chandigarh(U.T.)                   0.000000               0.000000
       Chattisgarh                        3.835990              19.932021
       Dadra & Nagar Haveli               1.633755               8.175905
       Delhi                              0.009241               0.146277
       Goa                                2.292424              10.585234
       Gujarat                            0.883754               5.419045
       Haryana                            0.736072               4.591903
       Himachal Pradesh                   2.526846               9.281232
       Jammu & Kashmir                    1.411953               6.207606
       Jharkhand                          2.997611              13.316326
       Karnataka                          1.815948               9.130688
       Kerala                             5.917278              26.378883
       Ladakh (U.T.)                      0.342992               2.826118
       Lakshadweep (U.T.)                 1.317188               4.160291
       Madhya Pradesh                     1.991260               9.375832
       Maharashtra                        0.763228               3.927712
       Manipur                            0.496880               3.986117
       Meghalaya                          2.909208              13.123720
       Mizoram                            0.076945               0.349433
       Nagaland                          12.792699              54.468782
       Odisha                            11.647648              51.931643
       Puducherry (U.T.)                  1.310784               8.268590
       Punjab                             1.102725               8.503563
       Rajasthan                          2.284589              12.119382
       Sikkim                             0.201413               0.604240
       Tamilnadu                          0.836909               4.823335
       Telangana                          0.247391               1.633835
       Tripura                            5.669061              37.511397
       Uttar Prdesh                       1.403836               7.702412
       Uttrakhand                         1.446670               6.368216
       West Bengal                        4.026102              20.790609

## Household average annual cylinder consumption

In [ ]:
# Across India

## Duplicate df
trial = df.copy()

## Sumproduct of consumption
value_mult = pd.Series({
    'lpg_qty': (trial['lpg_qty'] * trial['multiplier']).sum()/trial['multiplier'].sum(),
})

#Converting to annual cylinder consumption
value_mult = round(value_mult*12/14.2, 2)

display(value_mult)

lpg_qty    5.31
dtype: float64

In [ ]:
# Rural vs Urban

# Duplicate df and replace Nan values
trial = df.copy()
trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']] = trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']].fillna(0)


# Sumproduct of consumption
value_mult = trial.groupby('sector').apply(lambda table: pd.Series({
    'wt_lpg_qty': (table['lpg_qty']*table['multiplier']).sum()/table['multiplier'].sum(),
    }), include_groups = False)

#Converting to annual cylinder consumption
value_mult = round(value_mult*12/14.2, 2)

display(value_mult)

,wt_lpg_qty
sector,
rural,4.25
urban,7.63


In [34]:
# Rural vs Urban across deciles

# Duplicate df and replace Nan values
trial = df.copy()
trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']] = trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']].fillna(0)


# Sumproduct of consumption
value_mult = trial.groupby(['sector', 'decile'], observed=False).apply(lambda table: pd.Series({
    'wt_lpg_qty': (table['lpg_qty']*table['multiplier']).sum()/table['multiplier'].sum()
    }), include_groups = False).round(2)

#Converting to annual cylinder consumption
value_mult = round(value_mult*12/14.2, 2)

display(value_mult)

wt_lpg_qty
sector decile            
rural  1             1.47
       2             2.40
       3             3.00
       4             3.35
       5             3.91
       6             4.29
       7             4.75
       8             5.15
       9             5.67
       10            6.06
urban  1             6.70
       2             7.85
       3             8.26
       4             8.48
       5             8.46
       6             8.39
       7             8.21
       8             7.94
       9             7.45
       10            5.81

## Share of HH expenditure on LPG and firewood

In [ ]:
# Across India

## Duplicate df
trial = df.copy()

## Filtering for HHs with primary cooking source as firewood or LPG
trial = trial.loc[trial['source_cooking'].isin(['LPG','firewood and chips']),:]

## Replace NA values with 0
trial[['lpg_value', 'firewood_mkt_value']] = trial[['lpg_value', 'firewood_mkt_value']].fillna(0)

## Total expenditure
trial['monthly_totalexp'] = trial['hh_size']*trial['mpce']

## Checking groupsize
group_size = trial.groupby('source_cooking').size()
group_size.name = 'sample size'
display(pd.DataFrame(group_size))


## Sumproduct of consumption (Note: LPG and firewood consumption are recorded for last month. Therefore, no scaling required)
value_mult = trial.groupby('source_cooking').apply(lambda table: pd.Series({
    'wt_lpg_value': (table['lpg_value']*table['multiplier']).sum(),
    'wt_firewood_mkt_value': (table['firewood_mkt_value']*table['multiplier']).sum(),
    'wt_total_exp': (table['monthly_totalexp']*table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby('source_cooking').apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

# Average monthly per capita expenditure 
trial = value_mult.div(hh_size_mult, axis = 0)
trial = trial.rename(columns = {'wt_lpg_value': 'LPG MPCE','wt_firewood_mkt_value': 'Firewood mkt MPCE','wt_total_exp': 'Overall MPCE' })

display(trial)

,sample size
source_cooking,
LPG,160329
firewood and chips,86947


,LPG MPCE,Firewood mkt MPCE,Overall MPCE
source_cooking,,,
LPG,186.814535,10.213290,5413.970000
firewood and chips,36.005628,74.868513,3218.975518


In [9]:
# Across Rural vs Urban

## Duplicate df
trial = df.copy()

## Filtering for HHs with primary cooking source as firewood or LPG
trial = trial.loc[trial['source_cooking'].isin(['LPG','firewood and chips']),:]

## Replace NA values with 0
trial[['lpg_value', 'firewood_mkt_value']] = trial[['lpg_value', 'firewood_mkt_value']].fillna(0)

## Total expenditure
trial['monthly_totalexp'] = trial['hh_size']*trial['mpce']

## Checking groupsize
group_size = trial.groupby(['sector','source_cooking']).size()
group_size.name = 'sample size'
display(pd.DataFrame(group_size))


## Sumproduct of consumption (Note: LPG and firewood consumption are recorded for last month. Therefore, no scaling required)
value_mult = trial.groupby(['sector','source_cooking']).apply(lambda table: pd.Series({
    'wt_lpg_value': (table['lpg_value']*table['multiplier']).sum(),
    'wt_firewood_mkt_value': (table['firewood_mkt_value']*table['multiplier']).sum(),
    'wt_total_exp': (table['monthly_totalexp']*table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby(['sector','source_cooking']).apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

# Average monthly per capita expenditure 
trial = value_mult.div(hh_size_mult, axis = 0)
trial = trial.rename(columns = {'wt_lpg_value': 'LPG MPCE','wt_firewood_mkt_value': 'Firewood mkt MPCE','wt_total_exp': 'Overall MPCE' })

display(trial)

sample size
sector source_cooking                 
rural  LPG                       68824
       firewood and chips        79489
urban  LPG                       91505
       firewood and chips         7458

LPG MPCE  Firewood mkt MPCE  Overall MPCE
sector source_cooking                                                 
rural  LPG                 170.122620          15.799643   4535.333551
       firewood and chips   34.601772          74.288628   3196.014092
urban  LPG                 206.955572           3.472606   6474.162825
       firewood and chips   61.648604          85.460755   3638.391357

In [ ]:
# Rural across deciles 

## Duplicate df
trial = df.copy()

## Filtering for HHs with primary cooking source as firewood or LPG
trial = trial.loc[trial['source_cooking'].isin(['LPG','firewood and chips']),:]

## Replace NA values with 0
trial[['lpg_value', 'firewood_mkt_value']] = trial[['lpg_value', 'firewood_mkt_value']].fillna(0)

## Total expenditure
trial['monthly_totalexp'] = trial['hh_size']*trial['mpce']

## Checking groupsize
group_size = trial.groupby(['sector','source_cooking', 'decile'], observed = False).size()
group_size.name = 'sample size'
display(pd.DataFrame(group_size))


## Sumproduct of consumption (Note: LPG and firewood consumption are recorded for last month. Therefore, no scaling required)
value_mult = trial.groupby(['sector','source_cooking', 'decile'], observed = False).apply(lambda table: pd.Series({
    'wt_lpg_value': (table['lpg_value']*table['multiplier']).sum(),
    'wt_firewood_mkt_value': (table['firewood_mkt_value']*table['multiplier']).sum(),
    'wt_total_exp': (table['monthly_totalexp']*table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby(['sector','source_cooking', 'decile'], observed = False).apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

# Average monthly per capita expenditure 
trial = value_mult.div(hh_size_mult, axis = 0)
trial = trial.rename(columns = {'wt_lpg_value': 'LPG MPCE','wt_firewood_mkt_value': 'Firewood MPCE','wt_total_exp': 'Overall MPCE' })

display(trial)

sample size
sector source_cooking     decile             
rural  LPG                1              1395
                          2              2395
                          3              3320
                          4              4053
                          5              5127
                          6              6193
                          7              7573
                          8              9273
                          9             11932
                          10            17563
       firewood and chips 1              8641
                          2              8569
                          3              8634
                          4              8476
                          5              8224
                          6              8165
                          7              7785
                          8              7658
                          9              7092
                          10             6245
urban  LPG                1              4927
                          2              6640
                          3              7471
                          4              8161
                          5              8863
                          6              9621
                          7             10154
                          8             10846
                          9             11832
                          10            12990
       firewood and chips 1              2167
                          2              1414
                          3              1022
                          4               751
                          5               617
                          6               509
                          7               393
                          8               284
                          9               188
                          10              113

LPG MPCE  Firewood MPCE  Overall MPCE
sector source_cooking     decile                                         
rural  LPG                1       112.079725       9.903473   1653.685079
                          2       125.593447      13.926330   2121.000933
                          3       138.221295      13.526511   2455.685209
                          4       141.971726      14.426414   2774.248494
                          5       147.672545      15.685908   3098.957194
                          6       157.392539      16.572624   3461.219035
                          7       165.264728      16.053555   3893.935381
                          8       175.710741      17.256731   4464.332060
                          9       187.228222      16.224526   5373.347836
                          10      222.073353      16.862153   8501.308170
       firewood and chips 1        10.278343      65.620991   1568.063157
                          2        19.460962      69.300095   2109.551494
                          3        24.051983      71.842423   2453.106471
                          4        28.915245      73.892593   2765.178824
                          5        34.239857      77.412862   3091.058457
                          6        39.415235      77.849708   3449.593559
                          7        46.146606      77.357858   3880.498669
                          8        56.373968      81.645857   4450.597493
                          9        66.549785      80.206191   5322.926855
                          10       89.091601      83.025986   8186.491391
urban  LPG                1       147.135995       5.711964   2347.928692
                          2       167.385142       5.573784   3160.866999
                          3       180.583411       4.894925   3765.502022
                          4       187.609675       4.403429   4349.751354
                          5       196.253397       3.577699   4963.809160
                          6       208.487927       3.386093   5661.404252
                          7       216.067392       2.797072   6523.683621
                          8       228.569989       1.918707   7674.421273
                          9       242.821091       1.831113   9577.236534
                          10      280.609884       1.336495  16163.122656
       firewood and chips 1        33.249866      80.169621   2219.640038
                          2        55.801884      83.993024   3137.568537
                          3        71.317736      81.380764   3739.205017
                          4        75.697874      91.978074   4321.653201
                          5        88.113413      92.896199   4956.582687
                          6        99.218846      93.263910   5660.398067
                          7       121.196722     105.365837   6512.803843
                          8       131.819911      83.879849   7630.888652
                          9       139.526511     116.838684   9538.259293
                          10      163.706962     114.533243  15613.402205